## 1. Import Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

print("✓ Libraries imported successfully!")

✓ Libraries imported successfully!


## 2. Load Raw Data

In [2]:
# Load dataset
df_raw = pd.read_csv('gender_education_dataset.csv')

print(f"Raw dataset: {len(df_raw):,} rows, {df_raw['country'].nunique()} countries/regions")
print(f"Time range: {df_raw['year'].min()} - {df_raw['year'].max()}")

Raw dataset: 11,970 rows, 266 countries/regions
Time range: 1980 - 2024


## 3. Filter Aggregate Regions

In [3]:
# Define aggregate regions to exclude (these are not individual countries)
aggregate_regions = [
    'World', 'Arab World', 'Central Europe and the Baltics', 'East Asia & Pacific',
    'East Asia & Pacific (excluding high income)', 'East Asia & Pacific (IDA & IBRD countries)',
    'Euro area', 'Europe & Central Asia', 'Europe & Central Asia (excluding high income)',
    'Europe & Central Asia (IDA & IBRD countries)', 'European Union', 'Fragile and conflict affected situations',
    'Heavily indebted poor countries (HIPC)', 'High income', 'IBRD only', 'IDA & IBRD total',
    'IDA blend', 'IDA only', 'IDA total', 'Latin America & Caribbean',
    'Latin America & Caribbean (excluding high income)', 'Latin America & the Caribbean (IDA & IBRD countries)',
    'Least developed countries: UN classification', 'Low & middle income', 'Low income',
    'Lower middle income', 'Middle East & North Africa', 'Middle East & North Africa (excluding high income)',
    'Middle East & North Africa (IDA & IBRD countries)', 'Middle income', 'North America',
    'Not classified', 'OECD members', 'Other small states', 'Pacific island small states',
    'Post-demographic dividend', 'Pre-demographic dividend', 'Small states', 'South Asia',
    'South Asia (IDA & IBRD)', 'Sub-Saharan Africa', 'Sub-Saharan Africa (excluding high income)',
    'Sub-Saharan Africa (IDA & IBRD countries)', 'Upper middle income', 'Africa Eastern and Southern',
    'Africa Western and Central', 'Caribbean small states', 'Early-demographic dividend'
]

# Filter out aggregate regions
df = df_raw[~df_raw['country'].isin(aggregate_regions)].copy()

print(f"After filtering: {len(df):,} rows, {df['country'].nunique()} individual countries")
print(f"Removed: {len(df_raw) - len(df):,} aggregate region rows")

After filtering: 9,945 rows, 221 individual countries
Removed: 2,025 aggregate region rows


## 4. Filter Time Period (1980-2024)

In [4]:
# Focus on 1980-2024 for comprehensive analysis
df = df[df['year'] >= 1980].copy()

print(f"Filtered to 1980-2024: {len(df):,} rows")
print(f"Time span: {df['year'].max() - df['year'].min() + 1} years")

Filtered to 1980-2024: 9,945 rows
Time span: 45 years


## 5. Assign World Regions

Map countries to 7 world regions for regional analysis and imputation.

In [5]:
# Comprehensive region mapping
region_mapping = {
    # Sub-Saharan Africa
    'Angola': 'Sub-Saharan Africa', 'Benin': 'Sub-Saharan Africa', 'Botswana': 'Sub-Saharan Africa',
    'Burkina Faso': 'Sub-Saharan Africa', 'Burundi': 'Sub-Saharan Africa', 'Cameroon': 'Sub-Saharan Africa',
    'Cape Verde': 'Sub-Saharan Africa', 'Central African Republic': 'Sub-Saharan Africa', 'Chad': 'Sub-Saharan Africa',
    'Comoros': 'Sub-Saharan Africa', 'Congo, Dem. Rep.': 'Sub-Saharan Africa', 'Congo, Rep.': 'Sub-Saharan Africa',
    'Cote d\'Ivoire': 'Sub-Saharan Africa', 'Equatorial Guinea': 'Sub-Saharan Africa', 'Eritrea': 'Sub-Saharan Africa',
    'Eswatini': 'Sub-Saharan Africa', 'Ethiopia': 'Sub-Saharan Africa', 'Gabon': 'Sub-Saharan Africa',
    'Gambia, The': 'Sub-Saharan Africa', 'Ghana': 'Sub-Saharan Africa', 'Guinea': 'Sub-Saharan Africa',
    'Guinea-Bissau': 'Sub-Saharan Africa', 'Kenya': 'Sub-Saharan Africa', 'Lesotho': 'Sub-Saharan Africa',
    'Liberia': 'Sub-Saharan Africa', 'Madagascar': 'Sub-Saharan Africa', 'Malawi': 'Sub-Saharan Africa',
    'Mali': 'Sub-Saharan Africa', 'Mauritania': 'Sub-Saharan Africa', 'Mauritius': 'Sub-Saharan Africa',
    'Mozambique': 'Sub-Saharan Africa', 'Namibia': 'Sub-Saharan Africa', 'Niger': 'Sub-Saharan Africa',
    'Nigeria': 'Sub-Saharan Africa', 'Rwanda': 'Sub-Saharan Africa', 'Sao Tome and Principe': 'Sub-Saharan Africa',
    'Senegal': 'Sub-Saharan Africa', 'Seychelles': 'Sub-Saharan Africa', 'Sierra Leone': 'Sub-Saharan Africa',
    'Somalia': 'Sub-Saharan Africa', 'South Africa': 'Sub-Saharan Africa', 'South Sudan': 'Sub-Saharan Africa',
    'Sudan': 'Sub-Saharan Africa', 'Tanzania': 'Sub-Saharan Africa', 'Togo': 'Sub-Saharan Africa',
    'Uganda': 'Sub-Saharan Africa', 'Zambia': 'Sub-Saharan Africa', 'Zimbabwe': 'Sub-Saharan Africa',
    
    # Middle East & North Africa
    'Algeria': 'Middle East & North Africa', 'Bahrain': 'Middle East & North Africa', 'Djibouti': 'Middle East & North Africa',
    'Egypt, Arab Rep.': 'Middle East & North Africa', 'Iran, Islamic Rep.': 'Middle East & North Africa',
    'Iraq': 'Middle East & North Africa', 'Jordan': 'Middle East & North Africa', 'Kuwait': 'Middle East & North Africa',
    'Lebanon': 'Middle East & North Africa', 'Libya': 'Middle East & North Africa', 'Morocco': 'Middle East & North Africa',
    'Oman': 'Middle East & North Africa', 'Qatar': 'Middle East & North Africa', 'Saudi Arabia': 'Middle East & North Africa',
    'Syrian Arab Republic': 'Middle East & North Africa', 'Tunisia': 'Middle East & North Africa',
    'United Arab Emirates': 'Middle East & North Africa', 'West Bank and Gaza': 'Middle East & North Africa',
    'Yemen, Rep.': 'Middle East & North Africa',
    
    # Europe & Central Asia
    'Albania': 'Europe & Central Asia', 'Armenia': 'Europe & Central Asia', 'Austria': 'Europe & Central Asia',
    'Azerbaijan': 'Europe & Central Asia', 'Belarus': 'Europe & Central Asia', 'Belgium': 'Europe & Central Asia',
    'Bosnia and Herzegovina': 'Europe & Central Asia', 'Bulgaria': 'Europe & Central Asia', 'Croatia': 'Europe & Central Asia',
    'Cyprus': 'Europe & Central Asia', 'Czech Republic': 'Europe & Central Asia', 'Denmark': 'Europe & Central Asia',
    'Estonia': 'Europe & Central Asia', 'Finland': 'Europe & Central Asia', 'France': 'Europe & Central Asia',
    'Georgia': 'Europe & Central Asia', 'Germany': 'Europe & Central Asia', 'Greece': 'Europe & Central Asia',
    'Hungary': 'Europe & Central Asia', 'Iceland': 'Europe & Central Asia', 'Ireland': 'Europe & Central Asia',
    'Italy': 'Europe & Central Asia', 'Kazakhstan': 'Europe & Central Asia', 'Kosovo': 'Europe & Central Asia',
    'Kyrgyz Republic': 'Europe & Central Asia', 'Latvia': 'Europe & Central Asia', 'Lithuania': 'Europe & Central Asia',
    'Luxembourg': 'Europe & Central Asia', 'Moldova': 'Europe & Central Asia', 'Montenegro': 'Europe & Central Asia',
    'Netherlands': 'Europe & Central Asia', 'North Macedonia': 'Europe & Central Asia', 'Norway': 'Europe & Central Asia',
    'Poland': 'Europe & Central Asia', 'Portugal': 'Europe & Central Asia', 'Romania': 'Europe & Central Asia',
    'Russian Federation': 'Europe & Central Asia', 'Serbia': 'Europe & Central Asia', 'Slovak Republic': 'Europe & Central Asia',
    'Slovenia': 'Europe & Central Asia', 'Spain': 'Europe & Central Asia', 'Sweden': 'Europe & Central Asia',
    'Switzerland': 'Europe & Central Asia', 'Tajikistan': 'Europe & Central Asia', 'Turkiye': 'Europe & Central Asia',
    'Turkmenistan': 'Europe & Central Asia', 'Ukraine': 'Europe & Central Asia', 'United Kingdom': 'Europe & Central Asia',
    'Uzbekistan': 'Europe & Central Asia',
    
    # East Asia & Pacific
    'Australia': 'East Asia & Pacific', 'Brunei Darussalam': 'East Asia & Pacific', 'Cambodia': 'East Asia & Pacific',
    'China': 'East Asia & Pacific', 'Fiji': 'East Asia & Pacific', 'Indonesia': 'East Asia & Pacific',
    'Japan': 'East Asia & Pacific', 'Kiribati': 'East Asia & Pacific', 'Korea, Dem. People\'s Rep.': 'East Asia & Pacific',
    'Korea, Rep.': 'East Asia & Pacific', 'Lao PDR': 'East Asia & Pacific', 'Malaysia': 'East Asia & Pacific',
    'Marshall Islands': 'East Asia & Pacific', 'Micronesia, Fed. Sts.': 'East Asia & Pacific', 'Mongolia': 'East Asia & Pacific',
    'Myanmar': 'East Asia & Pacific', 'Nauru': 'East Asia & Pacific', 'New Zealand': 'East Asia & Pacific',
    'Palau': 'East Asia & Pacific', 'Papua New Guinea': 'East Asia & Pacific', 'Philippines': 'East Asia & Pacific',
    'Samoa': 'East Asia & Pacific', 'Singapore': 'East Asia & Pacific', 'Solomon Islands': 'East Asia & Pacific',
    'Thailand': 'East Asia & Pacific', 'Timor-Leste': 'East Asia & Pacific', 'Tonga': 'East Asia & Pacific',
    'Tuvalu': 'East Asia & Pacific', 'Vanuatu': 'East Asia & Pacific', 'Vietnam': 'East Asia & Pacific',
    
    # South Asia
    'Afghanistan': 'South Asia', 'Bangladesh': 'South Asia', 'Bhutan': 'South Asia',
    'India': 'South Asia', 'Maldives': 'South Asia', 'Nepal': 'South Asia',
    'Pakistan': 'South Asia', 'Sri Lanka': 'South Asia',
    
    # Latin America & Caribbean
    'Argentina': 'Latin America & Caribbean', 'Aruba': 'Latin America & Caribbean', 'Bahamas, The': 'Latin America & Caribbean',
    'Barbados': 'Latin America & Caribbean', 'Belize': 'Latin America & Caribbean', 'Bolivia': 'Latin America & Caribbean',
    'Brazil': 'Latin America & Caribbean', 'Chile': 'Latin America & Caribbean', 'Colombia': 'Latin America & Caribbean',
    'Costa Rica': 'Latin America & Caribbean', 'Cuba': 'Latin America & Caribbean', 'Curacao': 'Latin America & Caribbean',
    'Dominica': 'Latin America & Caribbean', 'Dominican Republic': 'Latin America & Caribbean', 'Ecuador': 'Latin America & Caribbean',
    'El Salvador': 'Latin America & Caribbean', 'Grenada': 'Latin America & Caribbean', 'Guatemala': 'Latin America & Caribbean',
    'Guyana': 'Latin America & Caribbean', 'Haiti': 'Latin America & Caribbean', 'Honduras': 'Latin America & Caribbean',
    'Jamaica': 'Latin America & Caribbean', 'Mexico': 'Latin America & Caribbean', 'Nicaragua': 'Latin America & Caribbean',
    'Panama': 'Latin America & Caribbean', 'Paraguay': 'Latin America & Caribbean', 'Peru': 'Latin America & Caribbean',
    'St. Kitts and Nevis': 'Latin America & Caribbean', 'St. Lucia': 'Latin America & Caribbean',
    'St. Vincent and the Grenadines': 'Latin America & Caribbean', 'Suriname': 'Latin America & Caribbean',
    'Trinidad and Tobago': 'Latin America & Caribbean', 'Uruguay': 'Latin America & Caribbean', 'Venezuela, RB': 'Latin America & Caribbean',
    
    # North America
    'Bermuda': 'North America', 'Canada': 'North America', 'United States': 'North America'
}

df['region'] = df['country'].map(region_mapping)

print(f"✓ Mapped {df['region'].notna().sum():,} rows to regions")
print(f"Unmapped countries: {df[df['region'].isna()]['country'].nunique()}")

✓ Mapped 8,415 rows to regions
Unmapped countries: 34


## 6. Hybrid Missing Value Imputation ⭐

**This is the CRITICAL step that was at the bottom of the old notebook!**

**Strategy:**
1. **Step 1:** Linear interpolation (temporal continuity within each country)
2. **Step 2:** Regional means (structural patterns across similar countries)
3. **Step 3:** KNN imputation (similarity-based for remaining gaps)

In [6]:
# Define indicator columns
indicator_cols = ['Girls_Out_Of_School_Primary', 'Literacy_Rate_Female', 'Literacy_Rate_Male', 
                  'Adolescent_Fertility_Rate', 'Female_Labor_Force_Participation']

# Start with clean dataset
df_processed = df.copy()

# Track original missing values
original_missing = df_processed[indicator_cols].isna().sum().sum()
total_cells = len(df_processed) * len(indicator_cols)

print("="*80)
print("HYBRID MISSING VALUE IMPUTATION")
print("="*80)
print(f"\nOriginal missing: {original_missing:,} / {total_cells:,} ({(original_missing / total_cells) * 100:.2f}%)")

# STEP 1: Linear Interpolation (temporal continuity within countries)
print("\nStep 1: Applying linear interpolation within each country...")
for country in df_processed['country'].unique():
    mask = df_processed['country'] == country
    country_data = df_processed[mask].sort_values('year')
    
    for col in indicator_cols:
        if country_data[col].notna().sum() >= 2:
            df_processed.loc[mask, col] = country_data[col].interpolate(
                method='linear', limit_direction='both', limit=10
            )

missing_after_step1 = df_processed[indicator_cols].isna().sum().sum()
print(f"   Filled: {original_missing - missing_after_step1:,} values")
print(f"   Remaining: {missing_after_step1:,} values")

# STEP 2: Regional Mean Imputation (structural patterns)
print("\nStep 2: Filling gaps with regional-year averages...")
for region in df_processed['region'].dropna().unique():
    for year in df_processed['year'].unique():
        mask = (df_processed['region'] == region) & (df_processed['year'] == year)
        
        for col in indicator_cols:
            regional_mean = df_processed.loc[mask, col].mean()
            missing_mask = mask & df_processed[col].isna()
            if not pd.isna(regional_mean):
                df_processed.loc[missing_mask, col] = regional_mean

missing_after_step2 = df_processed[indicator_cols].isna().sum().sum()
print(f"   Filled: {missing_after_step1 - missing_after_step2:,} additional values")
print(f"   Remaining: {missing_after_step2:,} values")

# STEP 3: KNN Imputation (similarity-based for remaining gaps)
print("\nStep 3: Applying KNN imputation for remaining gaps...")
if df_processed[indicator_cols].isna().sum().sum() > 0:
    for col in indicator_cols:
        if df_processed[col].isna().sum() > 0:
            pivot_data = df_processed.pivot_table(index=['country', 'year'], values=col, aggfunc='first')
            imputer = KNNImputer(n_neighbors=5, weights='distance')
            country_year_idx = pivot_data.index
            values = pivot_data.values.reshape(-1, 1)
            imputed_values = imputer.fit_transform(values)
            imputed_dict = dict(zip(country_year_idx, imputed_values.flatten()))
            df_processed[col] = df_processed.apply(
                lambda row: imputed_dict.get((row['country'], row['year']), row[col]), axis=1
            )

final_missing = df_processed[indicator_cols].isna().sum().sum()
total_filled = original_missing - final_missing

print(f"   Filled: {missing_after_step2 - final_missing:,} additional values")
print(f"\n{'='*80}")
print(f"✓ IMPUTATION COMPLETE: {total_filled:,} filled ({(total_filled / original_missing * 100):.1f}% coverage)")
print(f"  Final coverage: {((total_cells - final_missing) / total_cells * 100):.1f}%")
print("="*80)

HYBRID MISSING VALUE IMPUTATION

Original missing: 27,789 / 49,725 (55.89%)

Step 1: Applying linear interpolation within each country...
   Filled: 15,127 values
   Remaining: 12,662 values

Step 2: Filling gaps with regional-year averages...
   Filled: 15,127 values
   Remaining: 12,662 values

Step 2: Filling gaps with regional-year averages...
   Filled: 8,035 additional values
   Remaining: 4,627 values

Step 3: Applying KNN imputation for remaining gaps...
   Filled: 8,035 additional values
   Remaining: 4,627 values

Step 3: Applying KNN imputation for remaining gaps...
   Filled: 0 additional values

✓ IMPUTATION COMPLETE: 23,162 filled (83.3% coverage)
  Final coverage: 90.7%
   Filled: 0 additional values

✓ IMPUTATION COMPLETE: 23,162 filled (83.3% coverage)
  Final coverage: 90.7%


## 7. Create Derived Features

In [7]:
# 1. Literacy Gap (Male - Female)
df_processed['Literacy_Gap'] = (
    df_processed['Literacy_Rate_Male'] - df_processed['Literacy_Rate_Female']
)

# 2. Gender Parity Index (Female/Male ratio)
df_processed['Literacy_Gender_Parity_Index'] = (
    df_processed['Literacy_Rate_Female'] / df_processed['Literacy_Rate_Male']
)

# 3. Girls Out of School (in millions)
df_processed['Girls_Out_Of_School_Millions'] = (
    df_processed['Girls_Out_Of_School_Primary'] / 1_000_000
)

# 4. Normalize indicators (0-1 scale) for composite index
scaler = MinMaxScaler()

for col in indicator_cols:
    mask = df_processed[col].notna()
    if mask.sum() > 0:
        scaled_col_name = f'{col}_Scaled'
        df_processed.loc[mask, scaled_col_name] = scaler.fit_transform(
            df_processed.loc[mask, [col]]
        )

# 5. Gender Equality Index (composite 0-100 scale)
# Higher values = better gender equality
df_processed['Gender_Equality_Index'] = (
    (df_processed['Literacy_Rate_Female'] / 100) * 0.4 +
    (df_processed['Female_Labor_Force_Participation'] / 100) * 0.3 +
    ((200 - df_processed['Adolescent_Fertility_Rate']) / 200) * 0.3
) * 100

print("\n" + "="*70)
print("DERIVED FEATURES CREATED")
print("="*70)
print("\n✓ Literacy_Gap (Male - Female %)")
print("✓ Literacy_Gender_Parity_Index (Female/Male ratio)")
print("✓ Girls_Out_Of_School_Millions")
print("✓ Scaled indicators (0-1 range)")
print("✓ Gender_Equality_Index (composite 0-100 scale)")

print(f"\nFinal dataset: {len(df_processed):,} rows × {len(df_processed.columns)} columns")


DERIVED FEATURES CREATED

✓ Literacy_Gap (Male - Female %)
✓ Literacy_Gender_Parity_Index (Female/Male ratio)
✓ Girls_Out_Of_School_Millions
✓ Scaled indicators (0-1 range)
✓ Gender_Equality_Index (composite 0-100 scale)

Final dataset: 9,945 rows × 17 columns


## 8. Save Cleaned Dataset

In [8]:
# Save cleaned dataset for EDA and visualization
output_file = 'gender_education_cleaned.csv'
df_processed.to_csv(output_file, index=False)

print("\n" + "="*80)
print("✓ CLEANED DATASET SAVED")
print("="*80)
print(f"\nFile: {output_file}")
print(f"Size: {df_processed.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"Rows: {len(df_processed):,}")
print(f"Columns: {len(df_processed.columns)}")
print(f"Countries: {df_processed['country'].nunique()}")
print(f"Time range: {df_processed['year'].min()}-{df_processed['year'].max()}")
print(f"Data coverage: {((total_cells - final_missing) / total_cells * 100):.1f}%")
print("\n✓ Ready for Notebook 3: Exploratory Data Analysis")


✓ CLEANED DATASET SAVED

File: gender_education_cleaned.csv
Size: 2.38 MB
Rows: 9,945
Columns: 17
Countries: 221
Time range: 1980-2024
Data coverage: 90.7%

✓ Ready for Notebook 3: Exploratory Data Analysis


## Summary

### Data Cleaning Steps Completed:

1. ✓ **Filtered aggregate regions** (~45 aggregate regions removed)
2. ✓ **Filtered time period** (1980-2024, 45 years)
3. ✓ **Assigned world regions** (7 regions for analysis and imputation)
4. ✓ **Hybrid imputation applied:**
   - Linear interpolation within countries
   - Regional-year means
   - KNN imputation for remaining gaps
   - Achieved 80%+ data coverage
5. ✓ **Created derived features:**
   - Literacy gap and parity index
   - Scaled indicators
   - Composite gender equality index
6. ✓ **Saved cleaned dataset** (`gender_education_cleaned.csv`)

---

### Next Steps:

**→ Notebook 3:** Load cleaned data and perform exploratory data analysis with visualizations  
**→ Notebook 4:** Create polished interactive visualizations and generate insights

**Key Achievement:** The hybrid imputation (formerly at the bottom of the old notebook) is now properly placed BEFORE any analysis!